In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import st_folium
import plotly.express as px
import altair as alt
import plotly.graph_objects as go



In [35]:
# --- 1. Page Setup ---
st.set_page_config(page_title="India Cultural Experience Explorer", layout="wide")

# --- 2. Title and Intro ---
st.title("🇮🇳 India’s Cultural, Artistic & Heritage Explorer")

st.markdown("""
Welcome to the **India Cultural Experience Explorer** — your interactive gateway to the **art, heritage, and cultural richness** of India.

🎨 **What can you explore?**
- 📍 Discover famous monuments, art centers, and cultural events across Indian states.
- 📆 Understand the best times to visit using climate data, tourist season insights, and festivals.
- 🌱 Get personalized recommendations for responsible travel.

Start by selecting the **states** and **months** you’re interested in to tailor the report to your preferences.
""")

2025-05-16 16:04:30.565 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.567 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.568 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [36]:

# Load datasets
df_culture = pd.read_csv("datasets/cultural_sites.csv", encoding='windows-1252')
df_weather = pd.read_csv("datasets/weather_data.csv")

df_culture = df_culture.dropna(subset=['latitude', 'longitude'])


In [37]:
# Get unique states and months from both datasets
states = pd.concat([df_culture['state'], df_weather['state']]).dropna().unique()
months = df_weather['month'].dropna().unique()  # or just use your predefined all_months list

In [38]:
print(states)

['Uttar Pradesh' 'Kerala' 'Tamil Nadu' 'Madhya Pradesh' 'Karnataka'
 'West Bengal' 'Gujarat' 'Odisha' 'Maharashtra' 'Chandigarh' 'Delhi'
 'Assam' 'Goa' 'Telangana' 'Rajasthan' 'Ladakh' 'Bihar' 'Chhattisgarh'
 'Himachal Pradesh' 'Jammu & Kashmir' 'Andhra Pradesh' 'Arunachal Pradesh'
 'Meghalaya' 'Jharkhand' 'Mizoram' 'Sikkim' 'Tripura' 'Uttarakhand']


In [39]:
print(months)

['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']


In [40]:
with st.sidebar:
    st.header("🎛️ Customize Your Exploration")

    selected_states = st.multiselect(
        "🗺️ Select State(s):", 
        sorted(states), 
        default=None, 
        placeholder="All States"
    )

    selected_months = st.multiselect(
        "📅 Select Month(s):",
        sorted(months),  # or all_months
        default=None,
        placeholder="All Months",
        key="month_selector"
    )


2025-05-16 16:04:30.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [41]:
# Apply the same filters to both datasets
if selected_states:
    df_culture = df_culture[df_culture['state'].isin(selected_states)]
    df_weather = df_weather[df_weather['state'].isin(selected_states)]

if selected_months:
    df_weather = df_weather[df_weather['month'].isin(selected_months)]


In [42]:
# --- 6. Show Filter Summary ---
st.markdown(f"""
### 🎯 Your Current View
- **Selected State(s):** {', '.join(selected_states) if selected_states else 'All'}
- **Selected Month(s):** {', '.join(selected_months) if selected_months else 'All'}
""")

# --- 7. Prompt for Next Sections ---
st.markdown("👇 Scroll down to explore detailed **maps, climate graphs**, and **seasonal recommendations** based on your choices.")


2025-05-16 16:04:30.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.601 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [43]:
# --- 4. Initialize Folium Map ---
m = folium.Map(location=[22.9734, 78.6569], zoom_start=5, tiles='CartoDB positron')
marker_cluster = MarkerCluster().add_to(m)

# --- 5. Add Markers for Selected Monuments ---
for _, row in df_culture.iterrows():
    # Assuming 'image_url' contains the relative or absolute path to the image
    # Add some inline styling to limit image width or height so popup looks good
    img_html = f'<img src="{row["image_url"]}" alt="{row["monument"]}" style="width:100%; max-height:120px; object-fit:cover; margin-bottom:8px;" />'

    html = f"""
    <div style="width:220px">
        {img_html}
        <h4>{row['monument']}</h4>
        <b>City:</b> {row['city']}<br>
        <b>State:</b> {row['state']}<br>
        <b>2023-24 Total Visitors:</b> {row['2023-24 total visitors']}<br>
        <b>% Domestic Growth:</b> {row['% domestic growth']}%
    </div>
    """
    popup = folium.Popup(html, max_width=250)
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup,
        icon=folium.Icon(color="green", icon="university", prefix="fa")
    ).add_to(marker_cluster)

# --- 6. Display Map in Streamlit ---
with st.container():
    st_data = st_folium(m, width=800, height=600)

st.markdown("""
    <style>
    iframe {
        height: 600px !important;
        min-height: 600px !important;
        max-height: 600px !important;
    }
    .element-container:has(iframe) {
        margin-bottom: 0px !important;
        padding-bottom: 0px !important;
        height: 600px !important;
        overflow: hidden;
    }
    </style>
""", unsafe_allow_html=True)




2025-05-16 16:04:30.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [44]:
# --- 7. Top 3 Most Visited Monuments (Filtered) ---
st.subheader("🏆 Top 3 Most Visited Monuments (2023-24)")

# Sort and select top 3 from filtered data
top_3_monuments = df_culture.sort_values('2023-24 total visitors', ascending=False).head(3)


for _, row in top_3_monuments.iterrows():
    col1, col2 = st.columns([1, 2])
    with col1:
        # Clean relative image path (remove leading slash/backslash if any)
        image_path = os.path.normpath(row['image_url'].lstrip('/\\'))

        if os.path.exists(image_path):
            st.image(image_path, width=150, caption=row['monument'])
        else:
            st.markdown("🖼️ *Image not found*")
    with col2:
        st.markdown(f"**{row['monument']}**")
        st.markdown(f"📍 {row['city']}, {row['state']}")
        st.markdown(f"👥 **{row['2023-24 total visitors']:,} visitors**")
        st.markdown(f"📈 Domestic Growth: {row['% domestic growth']}%")
    st.markdown("---")


2025-05-16 16:04:30.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [45]:

# Mapping state to capitals
state_to_capital = {
    "Andhra Pradesh": "Amaravati", "Arunachal Pradesh": "Itanagar", "Assam": "Dispur",
    "Bihar": "Patna", "Chhattisgarh": "Raipur", "Goa": "Panaji", "Gujarat": "Gandhinagar",
    "Haryana": "Chandigarh", "Himachal Pradesh": "Shimla", "Jharkhand": "Ranchi",
    "Karnataka": "Bengaluru", "Kerala": "Thiruvananthapuram", "Madhya Pradesh": "Bhopal",
    "Maharashtra": "Mumbai", "Manipur": "Imphal", "Meghalaya": "Shillong",
    "Mizoram": "Aizawl", "Nagaland": "Kohima", "Odisha": "Bhubaneswar",
    "Punjab": "Chandigarh", "Rajasthan": "Jaipur", "Sikkim": "Gangtok",
    "Tamil Nadu": "Chennai", "Telangana": "Hyderabad", "Tripura": "Agartala",
    "Uttar Pradesh": "Lucknow", "Uttarakhand": "Dehradun", "West Bengal": "Kolkata"
}

# Month mappings
month_full_to_short = {
    "January": "Jan", "February": "Feb", "March": "Mar", "April": "Apr", "May": "May",
    "June": "Jun", "July": "Jul", "August": "Aug", "September": "Sep",
    "October": "Oct", "November": "Nov", "December": "Dec"
}
month_order = list(month_full_to_short.values())

# Add capital city
df_weather["capital"] = df_weather["state"].map(state_to_capital)

# Normalize month column
df_weather["month"] = pd.Categorical(df_weather["month"], categories=month_order, ordered=True)

# --- Streamlit UI ---
st.title("🌤️ Weather Dashboard for Indian States")

# Map selected full month names to short ones
selected_months_short = [month_full_to_short[m] for m in selected_months]

# Filter dataset
filtered_df = df_weather.copy()
if selected_states:
    filtered_df = filtered_df[filtered_df["state"].isin(selected_states)]
if selected_months_short:
    filtered_df = filtered_df[filtered_df["month"].isin(selected_months_short)]

# Rainfall classification
def rainfall_level(mm):
    if mm < 20:
        return "Low"
    elif mm < 100:
        return "Medium"
    else:
        return "High"

rainfall_colors = {
    "Low": "#a3d9a5",
    "Medium": "#f5de78",
    "High": "#f28c8c"
}

# Card generator
def generate_card(row):
    rain_level = rainfall_level(row["Rainfall (mm)"])
    rain_color = rainfall_colors[rain_level]
    return f"""
    <div style="
        background-color: #222;
        border-radius: 12px;
        padding: 15px;
        margin: 10px;
        color: white;
        box-shadow: 0 4px 6px rgba(0,0,0,0.3);
        font-family: Arial, sans-serif;
        width: 320px;
        flex-shrink: 0;
    ">
        <h3 style="margin-bottom: 8px;">{row['capital']}, {row['state']}</h3>
        <h4 style="margin-top: 0; margin-bottom: 12px;">Month: {row['month']}</h4>
        <p><strong>High:</strong> {row['Max Temperature (°C)']}°C | <strong>Low:</strong> {row['Min Temperature (°C)']}°C</p>
        <p style="
            background-color: {rain_color};
            color: #222;
            padding: 6px 10px;
            border-radius: 6px;
            font-weight: bold;
            display: inline-block;
        ">
            Rainfall: {row['Rainfall (mm)']} mm ({rain_level})
        </p>
    </div>
    """

# Generate cards
cards_per_row = 3
cards = [generate_card(row) for _, row in filtered_df.iterrows()]

# Display cards
if len(cards) == 0:
    st.warning("No data matches your filter selections.")
else:
    st.markdown("### ☁️ Weather Data Cards")
    for i in range(0, len(cards), cards_per_row):
        row_cards = cards[i:i + cards_per_row]
        row_html = "<div style='display: flex; flex-wrap: nowrap; justify-content: flex-start;'>" + "".join(row_cards) + "</div>"
        st.markdown(row_html, unsafe_allow_html=True)


2025-05-16 16:04:30.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:04:30.834 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [47]:
streamlit run get.ipynb

SyntaxError: invalid syntax (3622322547.py, line 1)